# Walmart Sales

## Import libraries

In [1]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score

## EDA

### Import data & basic statistics

In [2]:
df = pd.read_csv("src/Walmart_Store_sales.csv")

display(df.head())

display(df.describe(include='all'))

print("Types of each column :\n")
print(df.dtypes)
print("\n")

print("Missing value percentage :\n")
print(df.isnull().sum()/len(df)*100)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000


Types of each column :

Store           float64
Date             object
Weekly_Sales    float64
Holiday_Flag    float64
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
dtype: object


Missing value percentage :

Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64


- 'Date' feature needs to be converted. We willextract new features (year, month, day) from it.
- There are some missing values, but not enough to drop the columns. We will only drop missing values in the target ('weekly_update') column.
- 'Weekly_Sales' is the target, so we don't want to keep missing values in it.

In [3]:
# drop missing values in 'weekly_sales' column.
df = df.dropna(subset=['Weekly_Sales'])

In [4]:
#Extract new features from the Date column
df['Date'] = pd.to_datetime(df['Date'], format="%d-%m-%Y")
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Weekoftheyear'] = df['Date'].dt.isocalendar().week

#drop the Date column, as as it is no longer needed.
df = df.drop('Date', axis=1)

#### Outliers

Will be considered as outliers all the numeric features that don't fall within the range : $[Xˉ−3σ, Xˉ+3σ]$. 

In [5]:
num_features = ["Temperature", "Fuel_Price",'CPI','Unemployment']

fig = make_subplots(rows=1, cols=len(num_features))

for index, feature in enumerate(num_features,start=1):
    fig.add_trace(go.Box(y=df[feature], name=feature), row=1, col=index)
    
fig.update_layout(height=800, width=800)
fig.show()

In [6]:
#  Remove outliers
feature_list = ['Temperature','Fuel_Price','CPI','Unemployment']

for feature in feature_list:
    
    mean = df[feature].mean()
    std = df[feature].std()
    threshold = std * 3
    
    df_filtered = df[abs(df[feature]-mean) <= threshold]
    

## Analysis

In [27]:
fig = px.histogram(df, 
             x = 'Store',
             y = 'Weekly_Sales',
             height=500,
             width=900,
             nbins=20
             )
fig.show()

#### Target Distribution

In [37]:
df['Weekly_Sales']

0      1572117.54
1      1807545.43
3      1244390.03
4      1644470.66
5      1857533.70
          ...    
145    2248645.59
146     716388.81
147     845252.21
148     856796.10
149    1255087.26
Name: Weekly_Sales, Length: 136, dtype: float64

In [36]:
fig = px.histogram(df,
                   x = 'Weekly_Sales',
                   height=400,
                   width=600,
                   nbins=60)
fig.show()

#### Correlation matrix

In [10]:
corr_matrix = df.corr().round(2)

import plotly.figure_factory as ff

fig = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.tolist(),
                                  y = corr_matrix.index.tolist())
fig.update_layout(
    height=800,
    width=800
)


fig.show()


- There is a strong correlation between 'Year' and 'Fuel' features.
- There is a negative correlation between 'CPI' and 'Store' features.
- There is a perfect correlation between 'Month' and 'weekoftheyear'.


We will drop some features as they are strongly correlated to other features :
    - 'Fuel' seems to be less correlated to the target than 'Year'.
    - 'CPI' is moderately correlated to the target, we will keep it, and drop 'Store'.
    - Since we are using sales by week datas, we will keep 'weekoftheyear', and drop 'Month'.

## Training a linear regression model

In [11]:
df.head()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Weekoftheyear
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0,7
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0,12
3,11.0,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaN,NaN,NaN,<NA>
4,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0,21
5,4.0,1857533.70,0.0,NaN,2.756,126.160226,7.896,2010.0,5.0,28.0,21


In [12]:
X = df.drop(['Weekly_Sales','Fuel_Price','Month'], axis=1)
y = df['Weekly_Sales']

#Splitting the dataset
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [13]:
numeric_features = ['Temperature','CPI','Unemployment','Year','Day','Weekoftheyear']
categorical_features = ['Store','Holiday_Flag']


numeric_transformer = Pipeline(
    steps=[
        ("imputer",SimpleImputer(strategy="median")),  # missing values will be replaced by columns' median
        ("scaler", StandardScaler()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer",SimpleImputer(strategy="most_frequent")),  # missing values will be replaced by most frequent value
        ("encoder",OneHotEncoder(drop="first"))  # first column will be dropped to avoid creating correlations between features
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)


In [14]:

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [15]:
# Perform cross validation on the train set
regressor = LinearRegression()
scores = cross_val_score(regressor, X_train, y_train, cv=5)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())
print()

The cross-validated R2-score is :  0.935682191491645
The standard deviation is :  0.02763927045599709



The score of the model is high, and the standard deviation is low, which means that our model performs well with consistency on new data.

In [16]:
regressor = LinearRegression()

regressor.fit(X_train, y_train)

LinearRegression()

In [17]:
y_train_pred = regressor.predict(X_train)
y_test_pred = regressor.predict(X_test)

In [18]:
print("R2 score on training set : ", r2_score(y_train, y_train_pred))
print("R2 score on test set : ", r2_score(y_test, y_test_pred))

R2 score on training set :  0.9729999023726691
R2 score on test set :  0.9343401406342702


The score is high, which means our model perform wells. But there is a slightly difference between test and train score, indicating a bit of overfitting.

## Feature Importance

In [19]:
regressor.coef_

array([  -72373.56624589,    82939.85892997,  -105519.26721899,
         -38303.62931229,   -15860.1945563 ,    76948.02430325,
         420972.89724887, -1189965.74013751,   643940.81619768,
       -1358792.75372211,    61776.94640664, -1002286.77039294,
        -780347.77493424, -1093756.42525976,   607769.697392  ,
        -117361.47556213,     5908.64771192,   536834.14275387,
         587058.21293293,  -702912.56487854, -1105750.57374915,
        -669119.40261101,  -267784.90936658,    17154.46127434,
         403178.3461555 ,    41220.50320186])

In [20]:
column_names = []
for name, pipeline, features_list in preprocessor.transformers_: # loop over pipelines
    if name == 'num': # if pipeline is for numeric variables

        features = features_list # just get the names of columns to which it has been applied
    else: # if pipeline is for categorical variables
        features = pipeline.named_steps['encoder'].get_feature_names_out(features_list) # get output columns names from OneHotEncoder
    column_names.extend(features) # concatenate features names
        
print("Names of columns corresponding to each coefficient: ", column_names)

Names of columns corresponding to each coefficient:  ['Temperature', 'CPI', 'Unemployment', 'Year', 'Day', 'Weekoftheyear', 'Store_2.0', 'Store_3.0', 'Store_4.0', 'Store_5.0', 'Store_6.0', 'Store_7.0', 'Store_8.0', 'Store_9.0', 'Store_10.0', 'Store_11.0', 'Store_12.0', 'Store_13.0', 'Store_14.0', 'Store_15.0', 'Store_16.0', 'Store_17.0', 'Store_18.0', 'Store_19.0', 'Store_20.0', 'Holiday_Flag_1.0']


In [21]:
coefs = pd.DataFrame(index = column_names, data = regressor.coef_.transpose(), columns=["coefficients"])

# Compute abs() and sort values
feature_importance = abs(coefs).sort_values(by = 'coefficients')
feature_importance

,coefficients
Store_12.0,5.908648e+03
Day,1.586019e+04
Store_19.0,1.715446e+04
Year,3.830363e+04
Holiday_Flag_1.0,4.122050e+04
Store_6.0,6.177695e+04
Temperature,7.237357e+04
Weekoftheyear,7.694802e+04
CPI,8.293986e+04
Unemployment,1.055193e+05


In [22]:
fig = px.bar(feature_importance, orientation = 'v')
fig.update_layout(showlegend = False, 
                  margin = {'l': 120} # to avoid cropping of column names
                 )
fig.show()

the R2 score of the regression model indicates a tendency to overfit, so applying regularization is a good option.

When examining feature importance, we can observes that not all features significantly influence the prediction. In such cases, a Lasso Regression model could be a good solution.

## Regularization

### Lasso Model Performance

In [23]:
# Perform grid search
print("Grid search...")
regressor = Lasso()
# Grid of values to be tested
params = {
    'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 1]
}
gridsearch = GridSearchCV(regressor, param_grid = params, cv = 5, scoring='r2') 
gridsearch.fit(X_train, y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best R2 score : ", gridsearch.best_score_)


Grid search...
...Done.
Best hyperparameters :  {'alpha': 1}
Best R2 score :  0.9356959503146538


In [24]:
# Print R^2 scores
print("R2 score for the Lasso model on training set : ", gridsearch.score(X_train, y_train))
print("R2 score for the Lasso model on test set : ", gridsearch.score(X_test, y_test))


R2 score for the Lasso model on training set :  0.9729998972692646
R2 score for the Lasso model on test set :  0.9343421416822026


In [25]:
# Perform grid search
print("Grid search...")
regressor = Ridge()
# Grid of values to be tested
params = {
    'alpha': [0.1, 0.2, 0.3, 0.4, 0.5]
}
gridsearch = GridSearchCV(regressor, param_grid = params, cv = 5, scoring='r2') 
gridsearch.fit(X_train, y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best R2 score : ", gridsearch.best_score_)


Grid search...
...Done.
Best hyperparameters :  {'alpha': 0.1}
Best R2 score :  0.9341940958262605


In [26]:
# Print R^2 scores
print("R2 score for the Ridge model on training set : ", gridsearch.score(X_train, y_train))
print("R2 score for the Ridge model on test set : ", gridsearch.score(X_test, y_test))


R2 score for the Ridge model on training set :  0.971815184030207
R2 score for the Ridge model on test set :  0.9333737656755232


Regularization doesn't improve the model, and fail to correct the overfitting.
The model's overall score is good, regardless of the method used.